In [1]:
import numpy as np

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
    
class GF():
    def __init__(self, f):
        Max = np.max(f)
        f2 = np.zeros(Max+1, np.uint8)
        f2[f] = 1
        f = np.flip(f2)
        self.len_of_symbol = Max
        self.q = 2**self.len_of_symbol
        self.table = np.array(self.make_code_table(f))
        
    def sub_matrix(self, A, i, j):
        A1 = np.delete(A, i, 0)
        A2 = np.delete(A1, j, 1)
        return A2

    def det(self, A):
        size = A.shape
        if size[0]==2 and size[1]==2:
            d = self.add(self.multi(A[0, 0],A[1, 1]), self.multi(A[1, 0],A[0, 1]))
            return d
        else:
            d = 0
            for i in range(size[0]):
                d=self.add(self.multi(A[0, i],self.det(self.sub_matrix(A, 0, i))), d)
            return d
        
    def make_code_table(self, f):
        m = len(f)-1
        N = 2**m
        table = ["0"*m]
        x =""
        for i in range(1,m+1):
            if f[i]==0:
                x+="0"
            else:
                x+="1"
        for i in range(0, m):
            c="0"*m
            c = c[0:i] + "1"+c[i+1:]
            c = c[::-1]
            table.append(c)
        for i in range(m+1, N):
            a = table[-1]
            a1 = a[1:]+"0"
            b = "0"*m
            if a[0]=="1":
                b = x
            c = self.add_string_code(a1, b)
            table.append(c)
        return table

    def add_string_code(self, a, b):
        c =""
        for i in range(self.len_of_symbol):
            if a[i]!= b[i] :
                c+="1"
            else:
                c+="0"
        return c
    def add(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
            a= int(a)
            b = int(b)
            a1 = self.table[a]
            b1 = self.table[b]
            c1 = self.add_string_code(a1, b1)
            c, = np.where(self.table == c1)
            return c[0]
        elif not np.isscalar(a):
            if np.isscalar(b):
                raise ValueError("a, b must have same shape ( a:",a.shape,", b is scalar)" )
            else:
                a_size = a.shape
                b_size = b.shape
                if  len(a_size) ==2 and  len(a_size)==2:
                    if a_size[0] != b_size[0] or a_size[1] != b_size[1] :
                        raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )
                    else:
                        c = np.zeros_like(a, np.uint8)
                        for i in range(a_size[0]):
                            for j in range(a_size[1]):
                                c[i, j] = self.add(a[i, j], b[i, j])
                        return c
                elif len(a_size)==1 and len(b_size)==1 and a_size ==b_size :
                    c = np.zeros_like(a, np.uint8)
                    for i in range(a_size[0]):
                        c[i] = self.add(a[i], b[i])
                    return c
                else:
                    raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )         
    def multi(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
                if a ==0 or b == 0:
                    return 0
                else:
                    c = (a+b-2) % (self.q-1)
                    return c+1
        elif not np.isscalar(a):
            a_size = a.shape
            if np.isscalar(b):
                c = [self.multi(i, b) for i in a]
                return np.array(c)
            b_size = b.shape
            if a_size[1] != b_size[0] :
                raise ValueError("Can not multiply 2 matrix of dimension: ", a.shape, b.shape)
            else:
                c = np.zeros((a_size[0], b_size[1]), np.uint8)
                for i in range(a_size[0]):
                    for j in range(b_size[1]):
                        for k in range(a_size[1]):
                            c[i, j] =self.add(c[i, j], self.multi(a[i, k], b[k, j])) 
                return c
    def make_sub_matrix(self, x):
        N =  N = self.q -1
        sub = np.zeros((N, N), np.uint8)
        index = x-1
        for i in range(N):
            sub[i, index] = index+1
            index +=1
            if index ==N:
                index =0
        return sub
    def reserve_e(self, a):
            a2 = self.q+1-a
            if a2 == self.q:
                a2 = 1
            if a2 == self.q+1:
                a2 = 0
            return a2
    def Reserve_fast(self, X, show =False):
        N = X.shape[0]
        def make_I(n):
                In = np.zeros((n, n), np.uint8)
                for i in range(n):
                    In[i, i] =1
                return In
        X1 = make_I(N)
        for i in range(N):
            if show:
                print(i)
            C = make_I(N)
            for j in range(N):
                C[j, i] = self.multi(X[j, i], self.reserve_e(X[i,i]))
            C[i,i] = self.reserve_e(X[i,i])
            X = self.multi(C,X)
            X1 = self.multi(C,X1)
        return X1
    
def Resever(X, gl32):
    A = X.copy()
    size= A.shape
    if size[0]!=size[1]:
        print("Maxtrix is not square!")
        return None
    E = np.zeros_like(A)
    for i in range(E.shape[0]):
        E[i, i] =1
        
    def process_row( i, Ax, Ex):
        A = Ax.copy()
        E = Ex.copy()
        for j in range(i):
            if A[i, j]!=0:
                x = gl32.reserve_e(A[i, j])
                A[i] = gl32.multi(A[i], x)
                E[i] = gl32.multi(E[i], x)
                A[i] = gl32.add(A[i], A[j])
                E[i] =  gl32.add(E[i],E[j])
                
        if A[i, i]>1:
            x = gl32.reserve_e(A[i, i])
            A[i] = gl32.multi(A[i], x)
            E[i] = gl32.multi(E[i], x)
        return A, E
            
    # Convert to up triangle matrix
    i = 0
    stt = 0
    while i < size[0]:
        A, E = process_row(i, A, E)
        stt+=1
        if A[i, i]==0:
            if stt>size[0]-i+1:
                print("matrix is not ivertable")
                return 0
            A[i:-1], A[-1] = A[i+1:].copy(), A[i].copy()
            E[i:-1], E[-1] = E[i+1:].copy(), E[i].copy()
        else:
            stt= 0
            i+=1
            
    # Convert to E
    for i in range(size[0]):
        for j in range(i+1, size[1]):
            if A[i, j]!=0:
                for x in range(32):
                    if gl32.multi(x, 1)==A[i, j]:
                        A[i] = gl32.add(A[i] , gl32.multi(A[j], x))
                        E[i] = gl32.add(E[i] , gl32.multi(E[j], x))
                        break

    return E

def Rank(Zx, gl32):
    Z = Zx.copy()
    index = []
    i =-1
    stt = -1
    while i <np.min(Zx.shape)-1 and Z.shape[0]>=np.min(Zx.shape):
        i+=1
        stt+=1
        for j in range(i):
            if Z[i, j]!=0:
                x = 0
                for k in range(32):
                    if gl32.multi(k, Z[i, j])==Z[j, j]:
                        x = k
                        break  
                Z[i] = gl32.multi(Z[i], x)
                Z[i] = gl32.add(Z[i], Z[j])
        ok = True
        if Z[i, i]==0:
            ok = False
            for j in range(i+1, G):
                if Z[i, j]!=0:
                    Z[:,i], Z[:, j] = Z[:, j].copy(), Z[:, i].copy()
                    ok = True
                    break
        if ok:
            index.append(stt)
        else:
            Z = np.delete(Z, i, 0)
            i-=1
    return len(index), index

In [17]:
gl = GF([4,  1, 0])
N = gl.q-1

x = np.ones(N, np.uint8)
y = np.arange(1, N+1)
f0 = gl.add(x, y)
Q = np.zeros((N, N), np.uint8)
Q[0] = f0.copy()
for i in range(1, N):
    Q[i] = np.roll(gl.multi(Q[i-1], 2), 1)
    
    
N1 = 4
N2= 11


    
H_base = Q[:N1, N1:N2+N1]
print(Q)
print("#"*20)
print(H_base)
H = np.zeros((N1*N, N2*N), np.int16)
for i in range(N1):
    for j in range(N2):
        if H_base[i, j]!=0:
            sub_matrix = gl.make_sub_matrix(H_base[i, j])
            H[i*N:(i+1)*N, j*N:(j+1)*N] = sub_matrix
print(H[:25, :24])
# H = H[0:40]
np.save("H.npy", H)

[[ 0  5  9 15  2 11 14 10  3  8  6 13 12  7  4]
 [ 5  0  6 10  1  3 12 15 11  4  9  7 14 13  8]
 [ 9  6  0  7 11  2  4 13  1 12  5 10  8 15 14]
 [15 10  7  0  8 12  3  5 14  2 13  6 11  9  1]
 [ 2  1 11  8  0  9 13  4  6 15  3 14  7 12 10]
 [11  3  2 12  9  0 10 14  5  7  1  4 15  8 13]
 [14 12  4  3 13 10  0 11 15  6  8  2  5  1  9]
 [10 15 13  5  4 14 11  0 12  1  7  9  3  6  2]
 [ 3 11  1 14  6  5 15 12  0 13  2  8 10  4  7]
 [ 8  4 12  2 15  7  6  1 13  0 14  3  9 11  5]
 [ 6  9  5 13  3  1  8  7  2 14  0 15  4 10 12]
 [13  7 10  6 14  4  2  9  8  3 15  0  1  5 11]
 [12 14  8 11  7 15  5  3 10  9  4  1  0  2  6]
 [ 7 13 15  9 12  8  1  6  4 11 10  5  2  0  3]
 [ 4  8 14  1 10 13  9  2  7  5 12 11  6  3  0]]
####################
[[ 2 11 14 10  3  8  6 13 12  7  4]
 [ 1  3 12 15 11  4  9  7 14 13  8]
 [11  2  4 13  1 12  5 10  8 15 14]
 [ 8 12  3  5 14  2 13  6 11  9  1]]
[[ 0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0  0 

In [7]:
gl.make_sub_matrix(4)

array([[ 0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 11,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 14,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15],
       [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  3,  0,  0,  0,  0

In [5]:
q = gl.q
config = np.zeros((q, (q-1)//2, 2), np.int16)
stt = np.zeros(q, np.int16)
for i in range(1, q-1):
    for j in range(i+1, q):
        c = gl.add(i, j)
        config[c, stt[c]]=[i, j]
        stt[c]+=1
print(config)
np.save("config.npy", config)

[[[ 0  0]
  [ 0  0]
  [ 0  0]
  [ 0  0]
  [ 0  0]
  [ 0  0]
  [ 0  0]]

 [[ 2  5]
  [ 3  9]
  [ 4 15]
  [ 6 11]
  [ 7 14]
  [ 8 10]
  [12 13]]

 [[ 1  5]
  [ 3  6]
  [ 4 10]
  [ 7 12]
  [ 8 15]
  [ 9 11]
  [13 14]]

 [[ 1  9]
  [ 2  6]
  [ 4  7]
  [ 5 11]
  [ 8 13]
  [10 12]
  [14 15]]

 [[ 1 15]
  [ 2 10]
  [ 3  7]
  [ 5  8]
  [ 6 12]
  [ 9 14]
  [11 13]]

 [[ 1  2]
  [ 3 11]
  [ 4  8]
  [ 6  9]
  [ 7 13]
  [10 15]
  [12 14]]

 [[ 1 11]
  [ 2  3]
  [ 4 12]
  [ 5  9]
  [ 7 10]
  [ 8 14]
  [13 15]]

 [[ 1 14]
  [ 2 12]
  [ 3  4]
  [ 5 13]
  [ 6 10]
  [ 8 11]
  [ 9 15]]

 [[ 1 10]
  [ 2 15]
  [ 3 13]
  [ 4  5]
  [ 6 14]
  [ 7 11]
  [ 9 12]]

 [[ 1  3]
  [ 2 11]
  [ 4 14]
  [ 5  6]
  [ 7 15]
  [ 8 12]
  [10 13]]

 [[ 1  8]
  [ 2  4]
  [ 3 12]
  [ 5 15]
  [ 6  7]
  [ 9 13]
  [11 14]]

 [[ 1  6]
  [ 2  9]
  [ 3  5]
  [ 4 13]
  [ 7  8]
  [10 14]
  [12 15]]

 [[ 1 13]
  [ 2  7]
  [ 3 10]
  [ 4  6]
  [ 5 14]
  [ 8  9]
  [11 15]]

 [[ 1 12]
  [ 2 14]
  [ 3  8]
  [ 4 11]
  [ 5  7]
  [ 6 15]
  [ 

In [ ]:
M = H.shape[0]
N = H.shape[1]
G = 0
print(M, N, G)
T = Ht[:M-G,N-M+G:]
print(T)
T2 = Resever(T, gl32)
E = Ht[M-G:,N-M+G:]

In [ ]:
print(gl32.multi(T, T2))

In [ ]:
savetxt("Ht.txt",Ht.astype(np.uint8));
savetxt("T.txt",T.astype(np.uint8));
savetxt("E.txt",E.astype(np.uint8));

In [ ]:
X = Ht[:M-G, :N+G-M]
Y = Ht[M-G:, :N+G-M]
savetxt("X.txt",X.astype(np.uint8));
savetxt("Y.txt",Y.astype(np.uint8));
ET2 = gl32.multi(E, T2)
print(ET2)
ET2XY  = gl32.add(gl32.multi(ET2, X), Y)

In [ ]:
Z = ET2XY.transpose().copy()
savetxt("Z.txt",Z.astype(np.uint8));
rank , index = Rank(Z, gl32)

In [ ]:
print(index)

In [ ]:
Htd = Ht.copy()
Htd[:, N-M:N-M+G], Htd[:, index] = Ht[:, index], Ht[:, N-M:N-M+G]
savetxt("Htd.txt", Htd.astype(np.uint8));

In [ ]:
A = Htd[:M-G,:N-M]
B = Htd[:M-G, N-M:N-M+G]
C = Htd[M-G:, :N-M]
D = Htd[M-G:, N-M:N-M+G]
# np.save("Htd.npy", Htd)
# np.save("A.npy", A)
# np.save("B.npy", B)
# np.save("T.npy", T)
# np.save("C.npy", C)
# np.save("D.npy", D)
# np.save("E.npy", E)

np.savetxt("D.txt",D.astype(np.uint8),fmt='%i', delimiter=',');

In [ ]:
C2 =gl32.add(gl32.multi(gl32.multi(E, T2), A), C)
D2 =gl32.add(gl32.multi(gl32.multi(E, T2), B), D)
print(D2)
np.savetxt("D2.txt",D2.astype(np.uint8),fmt='%i', delimiter=',');
D3 = Resever(D2, gl32)
np.savetxt("D3.txt",D3.astype(np.uint8),fmt='%i', delimiter=',');
D3C2 = gl32.multi(D3, C2)
np.savetxt("D3C2.txt",D3C2.astype(np.uint8),fmt='%i', delimiter=',');

In [ ]:
BD3C2 = gl32.multi(B, D3C2)
ABD3C2 = gl32.add(A, BD3C2)
T2ABD3C2 = gl32.multi(T2,ABD3C2)

In [ ]:
u = np.random.randint(0, gl32.q, H.shape[1] - H.shape[0]).reshape(N-M, 1)
p1 = gl32.multi(D3C2, u)
Au = gl32.multi(A, u)
Bp1 = gl32.multi(B, p1)
p2 = gl32.multi(T2, gl32.add(Au, Bp1))
print(y0)
y0 = np.concatenate((u, p1, p2))
y = np.zeros_like(y0)
y[sub.astype(np.uint8)] = y0
print(y)
print("#"*30)
print(gl32.multi(H0, y))

In [ ]:
print(D3C2.shape)
print(T2ABD3C2.shape)
G = np.concatenate((D3C2, T2ABD3C2))
savetxt("G.txt", G)
p = gl32.multi(G, u)
y0 = np.concatenate((u, p))
y = np.zeros_like(y0)
y[sub.astype(np.uint8)] = y0
print(sub)
print(y)
print("#"*30)
print(gl32.multi(H0, y))

In [ ]:
np.save("G.npy", G)
np.save("sub.npy", sub.astype(np.uint8))
np.save("H.npy", H0)

In [ ]:
G2 = Resever(G, gl32)

In [ ]:
for row in G:
    S = "("
    for e in row:
        S+=str(e)
        S+=", "
    S = S[:-2]
    S+="),"
    print(S)

In [ ]:
s = "("
for i in u:
    s+=str(i[0])+", "
s+=")"
print(s)

In [ ]:
pi = 0
stt = 0
for i in range(726):
    pi=gl32.add(pi, gl32.multi(u[i][0], G[stt,i]))
    print(u[i][0], G[stt,i], pi)

In [ ]:
print(p)

In [ ]:
def nhan_hang(A, i, a):
    K = np.zeros_like(A)
    for j in range(A.shape[0]):
        K[j, j] = a if j==i-1 else 1
    return K

def doi_hang(A, i, j):
    K = np.zeros_like(A)
    for k in range(A.shape[0]):
        K[k, k] = 1
    K[i-1, i-1] = 0
    K[j-1, j-1] = 0
    K[i-1, j-1]= 1
    K[j-1, i-1] = 1
    return K

def cong_hang(A, a, i, j):
    K = np.zeros_like(A)
    for k in range(A.shape[0]):
        K[k, k] = 1
    K[j-1, i-1] = a
    return K

In [ ]:
A = np.array([
    [2, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 2],
    [0, 1, 0, 2]
])
K1 = nhan_hang(A, 1, 31)
K = K1.copy()
A = gl32.multi(K1, A)
print(A)
print("#"*20)
K2 = cong_hang(A, 1, 2, 4)
K = gl32.multi(K2, K)
A = gl32.multi(K2, A)
print(A)
print("#"*20)
K3 = nhan_hang(A, 4, 31)
K = gl32.multi(K3, K)
A = gl32.multi(K3, A)
print(A)
print("#"*20)
K4 = cong_hang(A, 31, 3, 1)
K = gl32.multi(K4, K)
A = gl32.multi(K4, A)
print(A)
print("#"*20)
K5 = cong_hang(A, 1, 4, 1)
K = gl32.multi(K5, K)
A = gl32.multi(K5, A)
print(A)
print("#"*20)
K6 = cong_hang(A, 2, 4, 3)
K = gl32.multi(K6, K)
A = gl32.multi(K6, A)
print(A)
print("#"*20)


In [ ]:
A = np.array([
    [2, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 2],
    [0, 1, 0, 2]
])
A2 = gl32.multi(A, K)
print(K)
print(A2)

In [ ]:
print(gl32.table)

In [ ]:
print(gl32.add(2, 19))

In [ ]:
for a in range(32):
    print("#"*20)
    print(a)
    for i in range(32):
        for j in range(i+1, 32):
            if gl32.add(i,j)==a:
                print(i, "+", j)